
# 归一化层简介

## BN，LN，IN，GN，WS 从学术化上解释差异：

1. **BatchNorm**

batch方向做归一化，算NHW的均值，对小batchsize效果不好；BN主要缺点是对batchsize的大小比较敏感，由于每次计算均值和方差是在一个batch上，所以如果batchsize太小，则计算的均值、方差不足以代表整个数据分布；

计算batch中样本的每个c的均值方差并归一化

2. **LayerNorm**     

channel方向做归一化，算CHW的均值，主要对RNN作用明显；
计算每个样本内部所有c的方差均值，并归一化

3. **InstanceNorm**

一个channel内做归一化，算$H*W$的均值，用在风格化迁移；因为在图像风格化中，生成结果主要依赖于某个图像实例，所以对整个$batch$归一化不适合图像风格化中，因而对HW做归一化。可以加速模型收敛，并且保持每个图像实例之间的独立。

4. **GroupNorm**

将$channel$方向分$group$，然后每个group内做归一化，算$(C//G)HW$的均值；这样与$batchsize$无关，不受其约束。

5. **SwitchableNorm**

将BN、LN、IN结合，赋予权重，让网络自己去学习归一化层应该使用什么方法。

6. **Weight Standardization**

权重标准化，2019年约翰霍普金斯大学研究人员提出。

               

# Batch-Norm          

## **Batch-Norm原理**               

$$x：NCHW , [Batch size , Channels , Hight , Width]$$
$$x_{mean} = np.mean(x,axis = 0)$$
$$x_{sigma} = np.std(x,axis = 0)$$

Batch归一化会使参数搜索问题变得容易，使神经网络对超参数的选择更加稳定。在神经网络中，已知一些中间值，假设你有一些$m$个大小的$batch$的隐藏单元值$z^{(1)}$到$z^{(m)}$(加入隐藏层描述的写法是$z^{[l](m)}$)。      

**Batch-Norm方法如下**(其中$\gamma$和$\beta$是需要学习的参数):       
$$\mu=\frac{1}{m}\sum_i^m z^{(i)}\\
\sigma^2=\frac{1}{m}\sum_i^m (z^{(i)}-\mu)^2\\
z_{norm}^{(i)}=\frac{z^{(i)}-\mu}{\sqrt{\sigma^2+\varepsilon}}\\
\widetilde{z}^{(i)}=\gamma z_{norm}^{(i)}+beta$$       

注意$\gamma$和$\beta$的作用是，可以随意设置的平均值与方差，它会精确转化这个方程；假设使用$Sigmoid$方程作为激活函数，且原本的参数导致输入值仅仅将输出值限制在$Sigmoid$中间的线性阶段，那么设置平均值与方差将其扩大可以更好地利用全段的激活函数。       
除此之外，当$\gamma=\sqrt{\sigma^2+\varepsilon},\beta=\mu$时，$\widetilde{z}^{(i)}=\gamma z_{norm}^{(i)}$

## **Batch-Norm在神经网络中的拟合**             
1. 在神经网络中，Batch-Norm通常在激活函数前(每层的$\gamma,\beta$都是需要训练的参数——梯度下降等方法):

<img style="float: center;" src="course_2_pics/week_3_2.png" width=500 height=500>        

2. 由于归一化的操作，因此神经元的$b,Bias$实际上都会被减去，因此可以设为0    

3. 此处$z^{[l]}$的维度是$(nx,1)$，$nx$代表该层神经元的个数，即每个神经元进行一次Batch-Norm，它的输出是一个值，得到一个$(1,1)$的值后，纵向组合代表一个层的$z^{[l]}$

## Batch归一化如何起作用      
1. 和逻辑斯蒂回归中相同，把不同维度的值归一化后，相对**在同一个数量级上**(变成一个圆)           


2. 可以**使权重比你的网络更滞后或更深层**     


假设一个神经网络用于图像识别:     

假设你已经在所有黑猫的图像上训练了数据集，如果现在你要把该模型应用于有色猫，这种情况下，正面的例子不只是左边的黑猫，还有右边其它颜色的猫。

<img style="float: center;" src="course_2_pics/week_3_3.png" width=700 height=700>       

假设训练集的分布如左图，正例是红圈，反例是黑叉，但在测试集也许如左图。在左边训练得很好的模块，在右边的效果也许不会很好，即使存在运行都很好的同一个函数，但学习算法很难根据左图去发现绿色的决策边界。

<img style="float: center;" src="course_2_pics/week_3_4.png" width=500 height=500>  

**convariate shift**：

假设$q_1(x)$是测试集中一个样本点的概率密度，$q_0(x)$是训练集中一个样本点的概率密度。最终我们估计一个条件概率密度$p(y|x,\theta)$，它由$x$和一组参数$\theta=\{\theta_1，\theta_2...\theta_m\}$所决定。对于一组参数来说，对应$loss(\theta)$函数评估性能的好坏


综上，当我们找出在$q_0(x)$分布上最优的一组$\theta'$时，传统机器学习假设训练集和测试集是独立同分布的，即$q_0(x)$=$q_1(x)$，所以可以推出最优$\theta'$依然可以保证q1（x）最优。但现实当中这个假设往往不成立，伴随**新数据产生，老数据会过时**，当$q_0(x)$不再等于$q_1(x)$时，就被称作covariate shift         

从下图被遮住的部分看来，上一层的$a^{[l-1]}$可以看作是一组输入的特征值，当下图遮住的部分去掉会发现，这组输入值由上一层的参数$w^{[l]},b^{[l]}$来决定，那么Batch-Norm做的即**将上一层可能差异变化很大的输入值限定在一个0-1范围内使其不会偏移更多**，导致这一层的参数不那么依赖上一层的参数。Batch归一化减少了输入值改变的问题——**即使使输入分布改变了一些，在网络中输入值在层与层传递的过程中，其分布会改变得更少**。

<img style="float: center;" src="course_2_pics/week_3_5.png" width=500 height=500>  

3. **Batch-Norm的归一化作用**        

    Batch归一化中非直观的一件事是，**因为在mini-batch上计算的均值和方差，而不是在整个数据集上**，均值和方差有一些小的噪声。缩放的过程也有一些噪音，因为它是用有些噪音的均值和方差计算得出的。这使神经元对于前序的隐藏单元的依赖更小一下(与Dropout相同，Dropout使单元对个别的神经元依赖更小一些)。


## Batch Norm的缺点

1. 需要较大的batch以体现整体数据分布                


2. 训练阶段需要保存每个batch的均值和方差(**指数加权平均等方法**实时计算$\mu,\sigma^2$) ，以求出整体均值和方差在inference阶段使用            


3. 不适用于可变长序列的训练，如RNN    

# Layer Norm

同样有可学习参数$\gamma,\beta$，在channel方向进行归一化(即对同example同feature的不同dimension进行归一化)